In [213]:
import numpy as np
import imageio
from PIL import Image

import math
from math import fabs

In [214]:
ilosc_katow = 1
ilosc_emiterow = 400
l = 400 #odleglosc od emitera nr 1 do emitera nr ilosc_emiterow

### Wczytanie zdjęcia

In [215]:
image = imageio.imread('Zdjecia/Kolo.jpg')

### Rozmiar zdjęcia

In [216]:
image.shape

(400, 400, 3)

### Zapisanie wysokości i szerokości zdjęcia

In [217]:
n_x = len(image)
n_y = len(image[0])

n_x, n_y

(400, 400)

### Ogólny algorytm Bresenhama 

### Zapalanie piksela - rysowanie lini

In [218]:
def zapalPiksel(x,y, kolor="czarny"):
    if kolor=="czarny":
        wartosc = 255
    elif kolor=="bialy":
        wartosc=0
        
    #image_m[x][y] = wartosc
    image[x][y] = [wartosc, wartosc, wartosc]

In [219]:
def zczytajPiksel(nr_emitera, nr_kata, x, y):
    sinog[nr_emitera][nr_kata] += image[x][y][0]

### Algorytm

In [220]:
### Ogólny algorytm Bresenhama
###############
def algBresenhama(nr_emitera, nr_kata, x1,y1,x2,y2):
#K01: 	Jeżeli x1 ≤ x2, to kx ← 1, inaczej kx ← -1 	; określamy krok X od x1 do x2
    if(x1<=x2):
        kx = 1
    else:
        kx = -1
#K02: 	Jeżeli y1 ≤ y2, to ky ← 1, inaczej ky ← -1 	; określamy krok Y od y1 do y2
    if(y1<=y2):
        ky = 1
    else:
        ky = -1
#K03: 	dx ← |x2 - x1| 	; odległość pomiędzy x1 i x2
    dx = math.fabs(x2 - x1)
#K04: 	dy ← |y2 - y1| 	; odległość pomiędzy y1 i y2
    dy = math.fabs(y2 - y1)
#K05: 	Zapal piksel x1,y1 	; pierwszy piksel odcinka
    zapalPiksel(x1, y1)
#K06: 	Jeżeli dx < dy, idź do kroku K16 	; dla kątów > 45° wykonujemy wersję algorytmu z przestawionymi współrzędnymi
    if(dx < dy):
        #K16: 	e ← dy / 2 	; wersja algorytmu Bresenhama ze zamienionymi współrzędnymi x i y
        e = dy/2
        for i in range(int(dy)):
#K17: 	Powtarzaj dy razy kroki K18...K23 
#K18: 	    y1 ← y1 + ky
            y1 = y1 + ky
#K19: 	    e ← e - dx 
            e = e - dx
#K20: 	    Jeżeli e ≥ 0, idź do kroku K23 	 
            if(e>=0):
                l_op_sinog[nr_emitera][nr_kata] += 1
                zczytajPiksel(nr_emitera, nr_kata, x1,y1)
                zapalPiksel(x1, y1)
            else:
#K21: 	    x1 ← x1 + kx 	 
                x1 = x1 + kx
#K22: 	    e ← e + dy 	 
                e = e + dy
#K23: 	    Zapal piksel x1,y1 	 
                l_op_sinog[nr_emitera][nr_kata] += 1
                zczytajPiksel(nr_emitera, nr_kata, x1,y1)
                zapalPiksel(x1, y2)
    else:
#K07: 	e ← dx / 2 	; obliczamy wartość początkową wyrażenia błędu
        e = (dx/2)
#K08: 	Powtarzaj dx razy kroki K09...K14 	; rysujemy pozostałe piksele w pętli
        for i in range(int(dx)):
#K09: 	    x1 ← x1 + kx 	; przesuwamy się w odpowiednią stronę w kierunku szybkim
            x1 = x1 + kx
#K10: 	    e ← e - dy 	; obliczamy wyrażenie błędu
            e = e - dy
#K11: 	    Jeżeli e ≥ 0, idź do kroku K14 	; jeśli wyrażenie błędu jest nieujemne, pomijamy ruch w kierunku wolnym
            if(e>0):
                l_op_sinog[nr_emitera][nr_kata] += 1
                zczytajPiksel(nr_emitera, nr_kata, x1,y1)
                zapalPiksel(x1, y1)
            else:
#K12: 	    y1 ← y1 + ky 	; przesuwamy się w odpowiednią stronę w kierunku wolnym
                y1 = y1 + ky
#K13: 	    e ← e + dx 	; obliczamy nowe wyrażenie błędu
                e = e + dx
#K14: 	    Zapal piksel x1.y1 	; kolejny piksel odcinka
                l_op_sinog[nr_emitera][nr_kata] += 1
                zczytajPiksel(nr_emitera, nr_kata, x1,y1)
                zapalPiksel(x1, y1)

### Zapisanie obrazu z tablicy do wersji graficznej .png, oraz wyświetlenie

In [221]:
img = Image.fromarray(image, 'RGB')
img.save('my.png')
img.show()

### Sinogram

In [222]:
sinog = [ [ 0 for y in range( ilosc_katow ) ] for x in range( ilosc_emiterow ) ]
l_op_sinog = [ [ 0 for y in range( ilosc_katow ) ] for x in range( ilosc_emiterow ) ]
sinogram = [ [ 0 for y in range( ilosc_katow ) ] for x in range( ilosc_emiterow ) ]

In [234]:
x_em = [ None for x in range(ilosc_emiterow + 1)]
y_em = [ None for y in range(ilosc_emiterow + 1)]
#położenie początkowe emitera głównego

y_em[0] = int(n_x/2)
x_em[0] = 0

#położenie emiterów odbierających

odstep = int(n_y/(ilosc_emiterow-1))
dod = 0
for i in range(1, ilosc_emiterow+1):
    x_em[i] = n_y - 1
    y_em[i] = 0 + dod
    dod += odstep
    

In [224]:
x_em, y_em

([0,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
  399,
 

In [225]:
for em in range(ilosc_emiterow):
    algBresenhama(em, 0, x_em[0], y_em[0], x_em[em+1], y_em[em+1])

In [229]:
def norma_res_to_sinogram():
    for k in range(ilosc_katow):
        for e in range(ilosc_emiterow):
            val = int(round(sinog[e][k]/l_op_sinog[e][k]))
            sinogram[e][k] = val

In [230]:
norma_res_to_sinogram()

In [231]:
for i in range(ilosc_emiterow):
    sinogram[i] = sinogram[i] * 400

In [232]:
sinogram

[[55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55,
  55

In [233]:
sinogram_array = np.asarray(sinogram)
img = Image.fromarray(sinogram_array, '1')
img.save('my.png')
img.show()